Just to show the diff between graph metrics from other methods


In [1]:
import sys
sys.path.append("../../")
from pathlib import Path
root = Path("../../")

PosixPath('../..')

In [15]:
import importlib

import pyTVDN.utils
importlib.reload(pyTVDN.utils)
from pyTVDN.utils import load_pkl, save_pkl

from pathlib import Path
import numpy as np
from easydict import EasyDict as edict
import matplotlib.pyplot as plt
from scipy import signal
from scipy.stats import fisher_exact, norm, ttest_ind
import pickle
import seaborn as sns
from tqdm import tqdm
import numbers
import pandas as pd
%matplotlib inline

### Load data

In [16]:
resDir = root/"results"
dataDir = root/"data"

PosixPath('../../data')

In [7]:
def wUFn(Amat):
    eigVals, eigVecs = np.linalg.eig(Amat)
    r = np.sum(np.cumsum(np.abs(eigVals))/np.sum(np.abs(eigVals)) <= 0.8) + 1
    if (eigVals[r-1].imag + eigVals[r].imag ) == 0:
        r = r + 1
    wU = np.abs(np.matmul(eigVecs[:, :r], eigVals[:r]))
    return wU, r, eigVals[:r]

### DMD

In [36]:
filName = f"Ctrl_data_NWM_DMD.pkl"
with open(resDir/filName, "rb") as f:
    CtrlDMD = pickle.load(f)
filName = f"AD_data_NWM_DMD.pkl"
with open(resDir/filName, "rb") as f:
    ADDMD = pickle.load(f)

In [37]:
fsAll = []
for v in ADDMD:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsAD = fsAll;

In [38]:
fsAll = []
for v in CtrlDMD:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsCtrl = fsAll;

In [39]:
fsName = ["r", "tQ", "tL", "wU"]

Ys = np.concatenate([np.ones(len(fsAD)), np.zeros(len(fsCtrl))])
XsAD = []
XsCtrl = []
for fName in fsName:
    if isinstance(fsAD[0][fName], numbers.Number):
        cfAD = np.array([fs[fName] for fs in fsAD]).reshape(-1, 1)
        cfCtrl = np.array([fs[fName] for fs in fsCtrl]).reshape(-1, 1)
    else:
        cfAD = np.array([fs[fName] for fs in fsAD])
        cfCtrl = np.array([fs[fName] for fs in fsCtrl])
    
    XsAD.append(cfAD)
    XsCtrl.append(cfCtrl)
XsAD = np.concatenate(XsAD, axis=1)
XsCtrl = np.concatenate(XsCtrl, axis=1);

In [40]:
for ix, nam in enumerate(fsName[:-1]):
    vec_AD = XsAD[:, ix]
    vec_Ctrl = XsCtrl[:, ix]
    pval = ttest_ind(vec_AD, vec_Ctrl).pvalue
    print(
        f"{nam}: {np.mean(vec_AD)-np.mean(vec_Ctrl):.3f} "
        f"{pval:.3f}"
         )

r: 1.807 0.000
tQ: -0.176 0.002
tL: -0.004 0.479


### ND(TV)

In [41]:
filName = f"Ctrl_data_NWM_TV.pkl"
with open(resDir/filName, "rb") as f:
    CtrlTV = pickle.load(f)
filName = f"AD_data_NWM_TV.pkl"
with open(resDir/filName, "rb") as f:
    ADTV = pickle.load(f)

In [42]:
fsAll = []
for v in ADTV:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsAD = fsAll;

In [43]:
fsAll = []
for v in CtrlTV:
    fs = edict()
    fs.tC = v["ms"][0]
    fs.tL = v["ms"][1]
    fs.tQ = v["ms"][2]
    fs.C = v["ms"][3]
    fs.L = v["ms"][4]
    fs.Q = v["ms"][5]
    fs.wU, fs.r, eigVal = wUFn(v["Amat"])
    fs.absEigValMean = np.abs(eigVal).mean()
    fs.absImagEigValMaxMean = np.abs(eigVal.imag).max()
    fsAll.append(fs)
fsCtrl = fsAll;

In [44]:
fsName = ["r", "tQ", "tL", "wU"]
#fsName = ["r", "tQ", "tC", "tL", "wU"]
#fsName = ["r", "absImagEigValMaxMean", "tQ", "tC", "tL", "wU"]
Ys = np.concatenate([np.ones(len(fsAD)), np.zeros(len(fsCtrl))])
XsAD = []
XsCtrl = []
for fName in fsName:
    if isinstance(fsAD[0][fName], numbers.Number):
        cfAD = np.array([fs[fName] for fs in fsAD]).reshape(-1, 1)
        cfCtrl = np.array([fs[fName] for fs in fsCtrl]).reshape(-1, 1)
    else:
        cfAD = np.array([fs[fName] for fs in fsAD])
        cfCtrl = np.array([fs[fName] for fs in fsCtrl])
    
    XsAD.append(cfAD)
    XsCtrl.append(cfCtrl)
XsAD = np.concatenate(XsAD, axis=1)
XsCtrl = np.concatenate(XsCtrl, axis=1)
Xs = np.concatenate([XsAD, XsCtrl], axis=0);

In [45]:
for ix, nam in enumerate(fsName[:-1]):
    vec_AD = XsAD[:, ix]
    vec_Ctrl = XsCtrl[:, ix]
    pval = ttest_ind(vec_AD, vec_Ctrl).pvalue
    print(
        f"{nam}: {np.mean(vec_AD)-np.mean(vec_Ctrl):.3f} "
        f"{pval:.3f}"
         )

r: 1.182 0.000
tQ: 0.022 0.821
tL: -0.021 0.437


### Dynamic DMD

In [46]:
filName = f"AD_data_dynDMD_withgraph.pkl"
with open(resDir/filName, "rb") as f:
    fsAD = pickle.load(f)
    
filName = f"Ctrl_data_dynDMD_withgraph.pkl"
with open(resDir/filName, "rb") as f:
    fsCtrl = pickle.load(f)
    
# for abs imag max eig
filName = f"AD_data_dynDMD.pkl"
with open(resDir/filName, "rb") as f:
    fsAD_add = pickle.load(f)
    
filName = f"Ctrl_data_dynDMD.pkl"
with open(resDir/filName, "rb") as f:
    fsCtrl_add = pickle.load(f)
    
map_dict = ["tmC", "tmL", "tQ", "C", "L", "Q"]
freq = 120

120

In [47]:
fs_TVDMD = fsAD
fs_TVDMD_add = fsAD_add

fsAll = []
for ix in range(len(fs_TVDMD)):
    raw_fs = edict(fs_TVDMD[ix])
    add_fs = edict(fs_TVDMD_add[ix])
    fs = edict()
    
    fs.r = raw_fs.r
    fs.ncpts = raw_fs.ncpts
    fs.absEigValMean = raw_fs.absEigValMean
    fs.wUmeanAll = raw_fs.wUmeanAll
    fs.absImagEigValMaxMean = add_fs.absImagEigValMaxMean
    
    fs.var_C = np.var([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.var_L = np.var([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.var_Q = np.var([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    fs.mean_C = np.mean([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.mean_L = np.mean([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.mean_Q = np.mean([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    dwell = np.diff(np.concatenate([[0], raw_fs.ecpts, [7200]])/freq)
    max_dwell_idx = np.argsort(-dwell)[0]
    fs.top1_C = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmC")]
    fs.top1_L = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmL")]
    fs.top1_Q = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tQ")]
    fs.max_dwell = np.max(dwell)
    
    
    fsAll.append(fs)
    
fsAD_trans = fsAll;

In [48]:
fs_TVDMD = fsCtrl
fs_TVDMD_add = fsCtrl_add

fsAll = []
for ix in range(len(fs_TVDMD)):
    raw_fs = edict(fs_TVDMD[ix])
    add_fs = edict(fs_TVDMD_add[ix])
    fs = edict()
    
    fs.r = raw_fs.r
    fs.ncpts = raw_fs.ncpts
    fs.absEigValMean = raw_fs.absEigValMean
    fs.wUmeanAll = raw_fs.wUmeanAll
    fs.absImagEigValMaxMean = add_fs.absImagEigValMaxMean
    
    fs.var_C = np.var([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.var_L = np.var([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.var_Q = np.var([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    fs.mean_C = np.mean([metrics[map_dict.index("tmC")] for metrics in raw_fs.graph_metrics])
    fs.mean_L = np.mean([metrics[map_dict.index("tmL")] for metrics in raw_fs.graph_metrics])
    fs.mean_Q = np.mean([metrics[map_dict.index("tQ")] for metrics in raw_fs.graph_metrics])
    
    dwell = np.diff(np.concatenate([[0], raw_fs.ecpts, [7200]])/freq)
    max_dwell_idx = np.argsort(-dwell)[0]
    fs.top1_C = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmC")]
    fs.top1_L = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tmL")]
    fs.top1_Q = raw_fs.graph_metrics[max_dwell_idx][map_dict.index("tQ")]
    fs.max_dwell = np.max(dwell)
    
    
    fsAll.append(fs)
    
fsCtrl_trans = fsAll;

In [49]:
penalty = "l2"
Ys = np.concatenate([np.ones(len(fsAD_trans)), np.zeros(len(fsCtrl_trans))])
fsName = ["r", "ncpts",
          "mean_Q", "mean_L", 
          "var_Q",  "var_L",  
          "wUmeanAll" ]
XsAD = []
XsCtrl = []
for fName in fsName:
    if isinstance(fsAD_trans[0][fName], numbers.Number):
        cfAD = np.array([fs[fName] for fs in fsAD_trans]).reshape(-1, 1)
        cfCtrl = np.array([fs[fName] for fs in fsCtrl_trans]).reshape(-1, 1)
    else:
        cfAD = np.array([fs[fName] for fs in fsAD_trans])
        cfCtrl = np.array([fs[fName] for fs in fsCtrl_trans])
    
    XsAD.append(cfAD)
    XsCtrl.append(cfCtrl)
XsAD = np.concatenate(XsAD, axis=1)
XsCtrl = np.concatenate(XsCtrl, axis=1);


In [50]:
for ix, nam in enumerate(fsName[:-1]):
    vec_AD = XsAD[:, ix]
    vec_Ctrl = XsCtrl[:, ix]
    pval = ttest_ind(vec_AD, vec_Ctrl).pvalue
    print(
        f"{nam}: {np.mean(vec_AD)-np.mean(vec_Ctrl):.3f} "
        f"{pval:.3f}"
         )

r: 0.523 0.005
ncpts: 14.932 0.001
mean_Q: -0.004 0.864
mean_L: -0.023 0.099
var_Q: 0.027 0.005
var_L: 0.013 0.001
